## Sunrise and Sunset from latitude
Author : Manan Bordia\
Last Modified : 27 May 2022

### Calculate day of the year ###

In [17]:
function calcDay(day::Int, month::Int, year::Int)::Int
    N1::Int = floor(275 * month / 9)
    N2::Int = floor((month + 9) / 12)
    N3::Int = (1 + floor((year - 4 * floor(year / 4) + 2) / 3))
    N::Int = N1 - (N2 * N3) + day - 30
    return N
end

calcDay (generic function with 1 method)

### Convert Latitude to Hours and Calculate Approx Time

In [18]:
function calcApproxTime(N::Int, timeType::String, longitude::Float64)::Float64
    lngHour = longitude / 15
    
    if timeType == "sunrise"
        t = N + ((6 - lngHour) / 24)
    elseif timeType == "sunset"
        t = N + ((18 - lngHour) / 24)
    end
    return t
end

calcApproxTime (generic function with 1 method)

### Calculate Sun's mean anomaly

In [19]:
function calcMeanAnom(t::Float64)::Float64
    return M = (0.9856 * t) - 3.289
end

calcMeanAnom (generic function with 1 method)

### Calculate Mod for Float values

In [20]:
function calcMod(x::Number, n::Int)::Number
    return x - floor(x/n)*n
end

calcMod (generic function with 1 method)

### Calculate Sun's true longitude

In [21]:
function calcTrueLong(MeanAnom::Float64)
    L = MeanAnom + (1.916 * sind(MeanAnom)) + (0.020 * sind(2*MeanAnom)) + 282.634
    return calcMod(L, 360)   
end

calcTrueLong (generic function with 1 method)

 ### Calculate Sun's right ascension with quadrant tranformation

In [22]:
function calcRightAscen(L::Float64)::Float64
    RA = atand(0.91764 * tand(L))
    return calcMod(RA, 360)
end

function quadTransRightAscenInHours(L::Float64, RA::Float64)::Float64
    Lquad = floor(L/90) * 90
    RAquad = floor(RA/90) * 90
    RA = RA + (Lquad - RAquad)
    return RA/15
end

quadTransRightAscenInHours (generic function with 1 method)

### Calculate Sun's declination

In [23]:
function calcDecli(L::Float64)::NamedTuple{(:sinDecli, :cosDecli), Tuple{Float64, Float64}}
    sinDecli = 0.39782 * sind(L)
    cosDecli = cosd(asind(sinDecli))
    return (sinDecli = sinDecli, cosDecli = cosDecli)
end

calcDecli (generic function with 1 method)

### Calculate Sun's Local Hour Angle

In [24]:
function calcLocHourAngle(timeType::String, sinDecli::Float64, cosDecli::Float64, latitude::Float64)::Float64
    zenith = 90 + 50/60
    cosH = (cosd(zenith) - (sinDecli * sind(latitude))) / (cosDecli * cosd(latitude))

    if (cosH >  1 && timeType == "sunrise") 
        error("Sun never rises on this location on the specified date")  
    elseif (cosH < -1)
        error("Sun never sets on this location on the specified date")  
    end
    return cosH
end

calcLocHourAngle (generic function with 1 method)

### Calculate H and convert into hours

In [25]:
function calcHInHours(timeType::String, cosH::Float64)::Float64
    H = nothing
    if timeType == "sunrise"
        H = 360 - acosd(cosH)
    elseif timeType == "sunset"
        H = acosd(cosH)
    else
        error("Wrong timeType provided.")
    end
    return H/15
end

calcHInHours (generic function with 1 method)

### Calculate local mean time for sunrise or sunset

In [26]:
function calcLocMeanTime(t::Float64, RA::Float64, H::Float64)::Float64
    return T = H + RA - (0.06571 * t) - 6.622
end

calcLocMeanTime (generic function with 1 method)

### Get time in hours in UTC timezone 

In [27]:
function convertToUTCInHours(T::Float64, longitude::Float64)::Float64
    lngHour = longitude / 15
    UT = T - lngHour
    return calcMod(UT, 24)
end

convertToUTCInHours (generic function with 1 method)

## Functions to calculate sunrise or sunset time from coordinates and date

In [28]:
function calcSunriseOrSunset(timeType::String,latitude::Float64, longitude::Float64, day::Int, month::Int, year::Int)::Float64
    N = calcDay(day, month, year)
    t = calcApproxTime(N, timeType, longitude)
    M = calcMeanAnom(t)
    L = calcTrueLong(M)
    RA = calcRightAscen(L)
    RA = quadTransRightAscenInHours(L, RA)
    decli = calcDecli(L)
    cosH = calcLocHourAngle(timeType, decli.sinDecli, decli.cosDecli, latitude) 

    H = calcHInHours(timeType, cosH)
    T = calcLocMeanTime(t, RA, H)
    return UT = convertToUTCInHours(T, longitude)
end

function calcSunriseTime(latitude::Float64, longitude::Float64, day::Int, month::Int, year::Int)
     return calcSunriseOrSunset("sunrise", latitude, longitude, day, month, year)
end

function calcSunsetTime(latitude::Float64, longitude::Float64,  day::Int, month::Int, year::Int)
    return calcSunriseOrSunset("sunset", latitude, longitude, day, month, year)
end

calcSunsetTime (generic function with 1 method)

### Get time in h:m:s format from UTC hours

In [29]:
function getTimeFromHours(t::Float64)
    t = calcMod(t,24)
    meridian = t >= 12 ? "pm" : "am"
    t = calcMod(t, 12)
    h :: Int = floor(t)
    m :: Int = floor((t - h)*60)
    s :: Int = floor(((t - h)*60 - m)*60)
    return (string(h) * ":" * string(m) * ":" * string(s) * " " * meridian)
end

getTimeFromHours (generic function with 1 method)

## Testing Code for Kota, Rajasthan (India) : 27 May 2022
Actual sunrise - 5:37 am \
Actual sunset - 7:10 pm

In [30]:
sunriseTime = calcSunriseTime(25.213816, 75.864754, 27,5,2022)
sunsetTime = calcSunsetTime(25.213816, 75.864754, 27,5,2022)

IST = +(5 + 30/60) # +5:30 hrs

println("Sunrise in UTC (hrs) - ", getTimeFromHours(sunriseTime))
println("Sunset in UTC (hrs) - ",getTimeFromHours(sunsetTime), '\n')

println("Sunrise in IST (hrs) - ",getTimeFromHours(sunriseTime + IST))
println("Sunset in IST (hrs) - ",getTimeFromHours(sunsetTime + IST))

Sunrise in UTC (hrs) - 0:7:26 am
Sunset in UTC (hrs) - 1:39:59 pm

Sunrise in IST (hrs) - 5:37:26 am
Sunset in IST (hrs) - 7:9:59 pm


## Testing Code for Boston, Massachusetts (USA) : 27 May 2022
Actual sunrise - 5:13 am\
Actual sunset - 8:10 pm

In [31]:
sunriseTime = calcSunriseTime(42.33823943197661, -71.08809895118287, 27,5,2022)
sunsetTime = calcSunsetTime(42.33823943197661, -71.08809895118287, 27,5,2022)
EDT = -4 # -4:00 hrs

println("Sunrise in UTC - ", getTimeFromHours(sunriseTime))
println("Sunset in UTC - ",getTimeFromHours(sunsetTime),'\n')

println("Sunrise in EDT - ",getTimeFromHours(sunriseTime + EDT))
println("Sunset in EDT - ",getTimeFromHours(sunsetTime + EDT))

Sunrise in UTC - 9:13:6 am
Sunset in UTC - 0:10:14 am

Sunrise in EDT - 5:13:6 am
Sunset in EDT - 8:10:14 pm


### Testing Code for Sydney, New South Wales (Australia) : 27 May 2022
Actual sunrise - 6:48 am \
Actual sunset - 4:56 pm

In [16]:
sunriseTime = calcSunriseTime(-33.866195962120834, 151.1808770597708, 27,5,2022)
sunsetTime = calcSunsetTime(-33.866195962120834, 151.1808770597708, 27,5,2022)
AEST = 10 # UTC+10:00 hrs

println("Sunrise in UTC - ", getTimeFromHours(sunriseTime))
println("Sunset in UTC - ",getTimeFromHours(sunsetTime),'\n')

println("Sunrise in AEST - ",getTimeFromHours(sunriseTime + AEST))
println("Sunset in AEST - ",getTimeFromHours(sunsetTime + AEST))

Sunrise in UTC - 8:48:8 pm
Sunset in UTC - 6:56:15 am

Sunrise in AEST - 6:48:8 am
Sunset in AEST - 4:56:15 pm
